In [1]:
import os
import io
import sys
from time import time

from PyQt5.uic import loadUi
from PyQt5 import uic
from PyQt5.QtWidgets import *
from PyQt5.QtGui import *
from PyQt5.QtCore import Qt
from PyQt5 import QtGui

#_*_coding:utf-8 _*_#
import pandas as pd
import numpy as np
from matplotlib.backends.backend_qt5agg import FigureCanvasQTAgg as FigureCanvas
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set()

from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

In [2]:
form_class = uic.loadUiType("C:\\Users\\thdek\\project\\ui\\jump_to_data.ui")[0]

In [3]:
class MainWindow(QMainWindow, form_class) :    
    def __init__(self) :
        super().__init__()
        self.setupUi(self)
        self.setWindowTitle("Jump to Data")
        self.setAcceptDrops(True)
        self.cwd = os.getcwd()
        ###################tab1_train data head정보 출력###################
        current_path = self.cwd + "\\titanic.csv"
        self.train = pd.read_csv(current_path)
        data_sample = self.train.head(10)
        self.pre_processing_df = self.train.copy() #Pre-Processing 후처리 위한 df copy
        r = list(data_sample)
        self.tab1_df.setColumnCount(len(data_sample.columns))
        self.tab1_df.setRowCount(len(data_sample.index))
        for i in range(len(data_sample.index)):
            for j in range(len(data_sample.columns)):
                self.tab1_df.setItem(i,j,QTableWidgetItem(str(data_sample.iloc[i, j])))
        self.tab1_df.setHorizontalHeaderLabels(r)
        ###################train.info 출력###################
        buf = io.StringIO()
        self.train.info(buf=buf)
        df_info = buf.getvalue()
        self.train_info.append(df_info)
        
        ###################Analysis pivot chart check_list구현###################
        self.attribute_list = self.train.columns.tolist() #피벗테이블 구현을 위한 컬럼 list 추출
        #x축 check_list 생성
        self.x_list = []
        self.y_list = []
        for i in self.attribute_list:
            cb_x = QCheckBox(i, self)
            self.verticalLayout.addWidget(cb_x)
            cb_x.stateChanged.connect(self.checkbox_x)

        #y축 check_list 생성
        for i in self.attribute_list:
            cb_y = QCheckBox(i, self)
            self.verticalLayout_2.addWidget(cb_y)
            cb_y.stateChanged.connect(self.checkbox_y)
        
        #chart submit button 생성
        self.pivot_submit.clicked.connect(self.pivot_chart)
        
        ###################Pre-Processing_checkbox###################
        self.attribute_box = []
        for i in self.attribute_list:
            cb_at = QCheckBox(i, self)
            self.horizontalLayout.addWidget(cb_at)
            cb_at.stateChanged.connect(self.checkbox_at)
        
        self.live_data()
        self.Insert.clicked.connect(self.Insert_data)
        self.del_btn.clicked.connect(self.del_data)
        self.Smoothing.clicked.connect(self.Smoothing_data)
        self.Sum.clicked.connect(self.sum_data)
        
        ###################sklearn start###################
        for i in self.attribute_list:
            self.rb_at = QRadioButton(i, self)
            self.Target_list.addWidget(self.rb_at)
            self.rb_at.clicked.connect(lambda:self.target(self.rb_at))
        self.ML_submit.clicked.connect(self.ML)
        
        ###################Progressbar###################
        self.progressBar.setValue(0)
        self.progressBar.setAlignment(Qt.AlignCenter)
        
        self.sample_btn.clicked.connect(self.sample)
        self.all_btn.clicked.connect(self.all_def)
        
        #livedata down
        self.live_down_btn.clicked.connect(self.live_down)
        os.system("pause")
    
    def download(url, file_name, self):
        request.urlretrieve(url, file_name)
    
    def sample(self):
        data_sample = self.train.head(10)
        r = list(data_sample)
        self.tab1_df.setColumnCount(len(data_sample.columns))
        self.tab1_df.setRowCount(len(data_sample.index))
        for i in range(len(data_sample.index)):
            for j in range(len(data_sample.columns)):
                self.tab1_df.setItem(i,j,QTableWidgetItem(str(data_sample.iloc[i, j])))
        self.tab1_df.setHorizontalHeaderLabels(r)
    
    def all_def(self):
        r = list(self.train)
        self.tab1_df.setColumnCount(len(self.train.columns))
        self.tab1_df.setRowCount(len(self.train.index))
        for i in range(len(self.train.index)):
            for j in range(len(self.train.columns)):
                self.tab1_df.setItem(i,j,QTableWidgetItem(str(self.train.iloc[i, j])))
        self.tab1_df.setHorizontalHeaderLabels(r)
        
    def target(self, state):
        self.radioBtn = self.sender()
        if self.radioBtn.isChecked() :
            self.rb_state = self.radioBtn.text()
        
    #check된 attribute값 sender        
    def checkbox_x(self, state):
        checkbox = self.sender()
        text_x = checkbox.text()
        if state == Qt.Checked:
            self.x_list.append(text_x)
        else :
            self.x_list.remove(text_x)
            
    def checkbox_y(self, state):
        checkbox = self.sender()
        text_y = checkbox.text()
        if state == Qt.Checked:
            self.y_list.append(text_y)
        else:
            self.y_list.remove(text_y)
    
    ###################chart 구성을 위한 데이터 처리###################     
    def pivot_chart(self):
        self.pivot_list = []
        self.train_re = self.train[self.x_list + self.y_list].copy() #사용자가 체크한 attribute만 df로 받음
        for i in self.y_list:
            self.label_y = self.train_re[i].values
            self.tmp_set = set(self.label_y)
            self.label_y = list(self.tmp_set) #중복제거
            for j in self.label_y:
                self.train_re[j] = self.train_re[i] #새로운 열 생성
                self.pivot_list = self.pivot_list + self.label_y
                self.mask = self.train_re[j].map(self.train_re[j]) == j
                self.train_re[j] = self.train_re[j].where(self.mask, None)
            del self.train_re[i]
        df = pd.pivot_table(self.train_re, index = self.x_list, values = self.pivot_list, aggfunc = 'count')
        df.plot(kind='bar')
        link_png = self.cwd + "\\img.png"
        plt.savefig(link_png, dpi=200, bbox_inches='tight')                     

        self.qPixmapFileVar = QPixmap()
        self.qPixmapFileVar.load(link_png)
        self.qPixmapFileVar = self.qPixmapFileVar.scaledToWidth(600)
        self.chart_lbl.setPixmap(self.qPixmapFileVar)
   
    ###################Pre-Processing 기능 구현###################
    def checkbox_at(self, state): #수정할 attribute text값 sending
        checkbox = self.sender()
        text_at = checkbox.text()
        if state == Qt.Checked:
            self.attribute_box.append(text_at)
        else :
            self.attribute_box.remove(text_at)
    
    #결측값 처리
    def Insert_data(self):
        if self.ins_combo.currentText() == "Average": #평균 대체
            for i in self.attribute_box:
                mean = self.pre_processing_df[i].mean()
                mean = np.rint(mean)
                self.pre_processing_df[i] = self.pre_processing_df[i].fillna(mean)
        elif self.ins_combo.currentText() == "Mode": #최빈값 대체
            for i in self.attribute_box:
                mode = self.pre_processing_df[i].mode()[0]
                self.pre_processing_df[i] = self.pre_processing_df[i].fillna(mode)
        elif self.ins_combo.currentText() == "Middle": #중위값 대체
            for i in self.attribute_box:
                median = self.pre_processing_df[i].median()
                self.pre_processing_df[i] = self.pre_processing_df[i].fillna(median)
        self.live_data()
    
    #Attribute 삭제
    def del_data(self):
        for i in self.attribute_box:
            self.pre_processing_df = self.pre_processing_df.drop(i, axis = 1)
        for i in reversed(range(self.horizontalLayout.count())): 
            self.horizontalLayout.itemAt(i).widget().deleteLater()
        self.attribute_list = [x for x in self.attribute_list if x not in self.attribute_box]
        self.attribute_box = []
        for i in self.attribute_list:
            cb_ab = QCheckBox(i, self)
            self.horizontalLayout.addWidget(cb_ab)
            cb_ab.stateChanged.connect(self.checkbox_at)
        self.live_data()
    
    #평활화(이상값 처리)    
    def Smoothing_data(self):  
        for i in self.attribute_box:
            outlier_list = self.IQR(self.pre_processing_df[i])
            mean = self.pre_processing_df[i].mean()
            mean = np.rint(mean)
            mode = self.pre_processing_df[i].mode()[0]
            median = self.pre_processing_df[i].median()
            for j in outlier_list:
                if self.smooth_combo.currentText() == "Average": #평균 대체
                    self.pre_processing_df[i] = list(map(lambda x : mean if x == j else x, self.pre_processing_df[i]))
                elif self.smooth_combo.currentText() == "Mode": #최빈값 대체                    
                    self.pre_processing_df[i] = list(map(lambda x : mode if x == j else x, self.pre_processing_df[i]))
                elif self.smooth_combo.currentText() == "Middle": #중위값 대체                
                    self.pre_processing_df[i] = list(map(lambda x : median if x == j else x, self.pre_processing_df[i]))
        self.live_data()
                
    def IQR(self, data):
        Q1 = np.percentile(data, 10)
        Q3 = np.percentile(data, 90)
        outlier_list = list(filter(lambda x : x < Q1 or Q3 < x, data))
        return outlier_list
        
    #차원축소
    def sum_data(self):
        text = self.sum_text.toPlainText()
        new_column = self.pre_processing_df[self.attribute_box[0]]
        for i in range(1, len(self.attribute_box)):
            new_column = self.pre_processing_df[self.attribute_box[0]].add(self.pre_processing_df[self.attribute_box[i]], fill_value=0)
        self.pre_processing_df[text] = new_column
        self.live_data()
    
    #Pre-processing result live dataset
    def live_data(self):
        r = list(self.pre_processing_df)
        self.tab2_df.setColumnCount(len(self.pre_processing_df.columns))
        self.tab2_df.setRowCount(len(self.pre_processing_df.index))
        for i in range(len(self.pre_processing_df.index)):
            for j in range(len(self.pre_processing_df.columns)):
                self.tab2_df.setItem(i,j,QTableWidgetItem(str(self.pre_processing_df.iloc[i, j])))
        self.tab2_df.setHorizontalHeaderLabels(r)
        
    def ML(self):
        self.feature_type()
        self.target_ML = self.pre_processing_df[self.rb_state]
        #Cross validation
        k = self.K_fold.value()
        k_fold = KFold(n_splits = k, shuffle = True, random_state = 0)
        self.ml_list = []
        self.accuracy_list = []
        self.classifier = 0
        if self.KNN.isChecked() == True: 
            self.classifier += 1
        if self.Decision_Tree.isChecked() == True :
            self.classifier += 1
        if self.Random_Forest.isChecked() == True :
            self.classifier += 1
        if self.Naive_Bayes.isChecked() == True :
            self.classifier += 1
        if self.SVM.isChecked() == True :
            self.classifier += 1
        self.progress = 100 / self.classifier
        self.total_progress = self.progress
        #KNN
        if self.KNN.isChecked() == True :
            self.knn_n = self.KNN_num.value()
            self.clf = KNeighborsClassifier(n_neighbors = self.knn_n)
            self.score = cross_val_score(self.clf, self.pre_processing_df, self.target_ML, cv = k_fold, n_jobs = 1, scoring = 'accuracy', verbose = 10)
            self.accuracy = round(np.mean(self.score)*100, 2)
            self.ml_list.append("KNN")
            self.accuracy_list.append(self.accuracy)
            self.progressBar.setValue(self.total_progress)
        #Decision_Tree
        if self.Decision_Tree.isChecked() == True :
            self.clf = DecisionTreeClassifier()
            self.score = cross_val_score(self.clf, self.pre_processing_df, self.target_ML, cv=k_fold, n_jobs=1, scoring='accuracy', verbose = 10)
            self.accuracy = round(np.mean(self.score)*100, 2)
            self.ml_list.append("Decision Tree")
            self.accuracy_list.append(self.accuracy)
            self.total_progress += self.progress
            self.progressBar.setValue(self.total_progress)
        #Random Forest
        if self.Random_Forest.isChecked() == True :
            self.tree = self.Random_Forest_num.value()
            self.clf = RandomForestClassifier(n_estimators=self.tree)
            self.score = cross_val_score(self.clf, self.pre_processing_df, self.target_ML, cv=k_fold, n_jobs=1, scoring='accuracy', verbose = 10)
            self.accuracy = round(np.mean(self.score)*100, 2)
            self.ml_list.append("Random Forest")
            self.accuracy_list.append(self.accuracy)
            self.total_progress += self.progress
            self.progressBar.setValue(self.total_progress)
        #Naive Bayes
        if self.Naive_Bayes.isChecked() == True :  
            self.clf = GaussianNB()
            self.score = cross_val_score(self.clf, self.pre_processing_df, self.target_ML, cv=k_fold, n_jobs=1, scoring='accuracy', verbose = 10)
            self.accuracy = round(np.mean(self.score)*100, 2)
            self.ml_list.append("Naive Bayes")
            self.accuracy_list.append(self.accuracy)
            self.total_progress += self.progress
            self.progressBar.setValue(self.total_progress)
        #SVM
        if self.SVM.isChecked() == True :
            self.clf = SVC()
            self.score = cross_val_score(self.clf, self.pre_processing_df, self.target_ML, cv=k_fold, n_jobs=1, scoring='accuracy', verbose = 10)
            self.accuracy = round(np.mean(self.score)*100, 2)
            self.ml_list.append("SVM")
            self.accuracy_list.append(self.accuracy)
        #Max값 탐지
        ML_max = self.accuracy_list.index(max(self.accuracy_list))
        cl_max_list = [self.ml_list[ML_max]]
        for i in range(len(self.accuracy_list)):
            if ML_max == i:
                continue
            elif self.accuracy_list[ML_max] == self.accuracy_list[i]:
                cl_max_list.append(self.ml_list[i])
        self.Accuracy_best.setText("Best Accuracy : " + str(self.accuracy_list[ML_max]))
        for i in cl_max_list:
            self.accuracy_list.remove(self.accuracy_list[ML_max])
            self.ml_list.remove(i)
        self.accuracy_txt = ', '.join(cl_max_list)
        self.classifier_lbl.setText(self.accuracy_txt)
        oth = QLabel("Others", self)
        self.others.addWidget(oth)
        for i in range(len(self.accuracy_list)):
            str_ml = "Accuracy : " + str(self.accuracy_list[i]) + ", " + str(self.ml_list[i]) + "     "
            accuracy_lb = QLabel(str_ml, self)
            self.Accuracy_layout.addWidget(accuracy_lb)
        self.progressBar.setValue(100)
    def feature_type(self):
        for i in self.attribute_list:
            if self.pre_processing_df[i].dtype == "int64":
                continue
            elif self.pre_processing_df[i].dtype == "float64":
                continue
            else:
                value_list = self.pre_processing_df[i].values
                value_set = set(value_list)
                value_list = list(value_set)
                print(value_list)
                for j in value_list:
                    self.pre_processing_df[i] = list(map(lambda x : 1/(value_list.index(j)+1) if x == j else x, self.pre_processing_df[i]))
                print(self.pre_processing_df[i])
    def dragEnterEvent(self, event):
        if event.mimeData().hasUrls():
            event.accept()
        else:
            event.ignore()
    
    def dragMoveEvent(self, event):
        if event.mimeData().hasUrls():
            event.setDropAction(Qt.CopyAction)
            event.accept()
        else:
            event.ignore()

    def dropEvent(self, event):
        self.files = [u.toLocalFile() for u in event.mimeData().urls()]
        self.train = pd.read_csv(self.files[0], engine='python')
        ###################sample data_re###################
        self.pre_processing_df = self.train.copy() #Pre-Processing 후처리 위한 df copy
        data_sample = self.train.head(10)
        r = list(data_sample)
        self.tab1_df.setColumnCount(len(data_sample.columns))
        self.tab1_df.setRowCount(len(data_sample.index))
        for i in range(len(data_sample.index)):
            for j in range(len(data_sample.columns)):
                self.tab1_df.setItem(i,j,QTableWidgetItem(str(data_sample.iloc[i, j])))
        self.tab1_df.setHorizontalHeaderLabels(r)
        ###################train.info 변경###################
        buf = io.StringIO()
        self.train.info(buf=buf)
        df_info = buf.getvalue()
        self.train_info.clear()
        self.train_info.append(df_info)
        ###################Analysis pivot chart check_list 변경###################
        self.attribute_list = self.train.columns.tolist() #피벗테이블 구현을 위한 컬럼 list 추출
        #x축 check_list 생성
        self.x_list = []
        self.y_list = []
        #기존항목 제거
        for i in reversed(range(self.verticalLayout.count())): 
            self.verticalLayout.itemAt(i).widget().deleteLater()
        for i in reversed(range(self.verticalLayout_2.count())): 
            self.verticalLayout_2.itemAt(i).widget().deleteLater()
        for i in self.attribute_list:
            cb_x = QCheckBox(i, self)
            self.verticalLayout.addWidget(cb_x)
            cb_x.stateChanged.connect(self.checkbox_x)
        #y축 check_list 재생성
        for i in self.attribute_list:
            cb_y = QCheckBox(i, self)
            self.verticalLayout_2.addWidget(cb_y)
            cb_y.stateChanged.connect(self.checkbox_y)
        ###################Pre-Processing_checkbox###################
        #기존항목 제거
        for i in reversed(range(self.horizontalLayout.count())): 
            self.horizontalLayout.itemAt(i).widget().deleteLater()
        #새로운 항목 삽입
        for i in self.attribute_list:
            cb_at = QCheckBox(i, self)
            self.horizontalLayout.addWidget(cb_at)
            cb_at.stateChanged.connect(self.checkbox_at)        
        self.live_data()
        ###################target reset###################
        #기존항목 제거
        for i in reversed(range(self.Target_list.count())): 
            self.Target_list.itemAt(i).widget().deleteLater()
        #새로운 항목 삽입
        for i in self.attribute_list:
            self.rb_at = QRadioButton(i, self)
            self.Target_list.addWidget(self.rb_at)
            self.rb_at.clicked.connect(lambda:self.target(self.rb_at))
            
    def live_down(self):
        file_num = 1
        live_path = self.cwd + "\\current_data" + ".csv"
        while os.path.exists(live_path):
            live_path = self.cwd + "\\current_data" +"(" + str(file_num) + ")" + ".csv"
            file_num = file_num + 1
        self.pre_processing_df.to_csv(live_path)

In [4]:
if __name__ == "__main__" :
    #QApplication : 프로그램을 실행시켜주는 클래스
    app = QApplication(sys.argv) 

    #WindowClass의 인스턴스 생성
    myWindow = MainWindow() 

    #프로그램 화면을 보여주는 코드
    myWindow.show()

    #프로그램을 이벤트루프로 진입시키는(프로그램을 작동시키는) 코드
    app.exec_()